# Nasa weblogs exercises:

In [2]:
#imports:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
#sparkSession:
spark = (SparkSession.builder
    .master(master = "local")
    .appName(name = "curso")
    .config("spark.some.config.option", "some-value")
    .getOrCreate())
      
spark.sparkContext.setLogLevel("ERROR")

#### Load nasa weblogs  as dataframe:

In [13]:
weblogs = spark.read.option("sep"," ").option("header","true").option("inferSchema","true").csv("nasa_aug95.csv")

In [14]:
weblogs.show()

+--------------------+-------------------+--------------------+------+-------------+
|     requesting_host|           datetime|             request|status|response_size|
+--------------------+-------------------+--------------------+------+-------------+
|   in24.inetnebr.com|1995-08-01 00:00:01|GET /shuttle/miss...|   200|         1839|
|     uplherc.upl.com|1995-08-01 00:00:07|      GET / HTTP/1.0|   304|            0|
|     uplherc.upl.com|1995-08-01 00:00:08|GET /images/ksclo...|   304|            0|
|     uplherc.upl.com|1995-08-01 00:00:08|GET /images/MOSAI...|   304|            0|
|     uplherc.upl.com|1995-08-01 00:00:08|GET /images/USA-l...|   304|            0|
|ix-esc-ca2-07.ix....|1995-08-01 00:00:09|GET /images/launc...|   200|         1713|
|     uplherc.upl.com|1995-08-01 00:00:10|GET /images/WORLD...|   304|            0|
|slppp6.intermind.net|1995-08-01 00:00:10|GET /history/skyl...|   200|         1687|
|piweba4y.prodigy.com|1995-08-01 00:00:10|GET /images/launc...|  

In [15]:
weblogs.printSchema()

root
 |-- requesting_host: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- request: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- response_size: integer (nullable = true)



#### List the different kinds of web protocols in the dataframe:

In [22]:
protocols = (weblogs.withColumn("Protocol",regexp_extract(col("request"),"\s([A-Z]+/\w*.\d*)",1))
    .select("Protocol").where(col("Protocol").isNotNull()).distinct())

In [23]:
protocols.show()

+---------+
| Protocol|
+---------+
| HTML/1.0|
|         |
|HTTP/V1.0|
| HTTP/1.0|
+---------+



#### Count the ocurrences of the differents Status Codes:

In [24]:
status_count = weblogs.groupBy("status").count().orderBy(desc("count"))

In [25]:
status_count.show()

+------+-------+
|status|  count|
+------+-------+
|   200|1398988|
|   304| 134146|
|   302|  26497|
|   404|  10054|
|   403|    171|
|   501|     27|
|   500|      3|
|   400|      2|
+------+-------+



#### Count the ocurrence of all the differents types of requests in the dataframe:

In [26]:
request_count = weblogs.withColumn("request_type",regexp_extract(col("request"),"(\w*)\s/",1)).groupBy("request_type").count().orderBy(desc("count"))

In [27]:
request_count.show()

+------------+-------+
|request_type|  count|
+------------+-------+
|         GET|1565753|
|        HEAD|   3965|
|        POST|    111|
|            |     59|
+------------+-------+



#### Find the resource with the largest byte transfer:

In [29]:
weblog_resource = weblogs.withColumn("resource",regexp_extract(col("request"),"\s/([\S]*)",1))

In [30]:
weblog_resource.show()

+--------------------+-------------------+--------------------+------+-------------+--------------------+
|     requesting_host|           datetime|             request|status|response_size|            resource|
+--------------------+-------------------+--------------------+------+-------------+--------------------+
|   in24.inetnebr.com|1995-08-01 00:00:01|GET /shuttle/miss...|   200|         1839|shuttle/missions/...|
|     uplherc.upl.com|1995-08-01 00:00:07|      GET / HTTP/1.0|   304|            0|                    |
|     uplherc.upl.com|1995-08-01 00:00:08|GET /images/ksclo...|   304|            0|images/ksclogo-me...|
|     uplherc.upl.com|1995-08-01 00:00:08|GET /images/MOSAI...|   304|            0|images/MOSAIC-log...|
|     uplherc.upl.com|1995-08-01 00:00:08|GET /images/USA-l...|   304|            0|images/USA-logosm...|
|ix-esc-ca2-07.ix....|1995-08-01 00:00:09|GET /images/launc...|   200|         1713|images/launch-log...|
|     uplherc.upl.com|1995-08-01 00:00:10|GET 

In [32]:
largest_resource = weblog_resource.groupBy("resource").max("response_size").orderBy(desc("max(response_size)")).limit(1)

In [33]:
largest_resource.show()

+--------------------+------------------+
|            resource|max(response_size)|
+--------------------+------------------+
|statistics/1995/J...|           3421948|
+--------------------+------------------+



#### Most requested resource:

In [34]:
top_resource = weblog_resource.groupBy("resource").count().orderBy(desc("count")).limit(1)

In [35]:
top_resource.show()

+--------------------+-----+
|            resource|count|
+--------------------+-----+
|images/NASA-logos...|97410|
+--------------------+-----+



#### Day with most traffic:

In [36]:
bussiest_day = weblogs.withColumn("date",to_date("datetime")).groupBy("date").count().orderBy(desc("count")).limit(1)

In [37]:
bussiest_day.show()

+----------+-----+
|      date|count|
+----------+-----+
|1995-08-31|90122|
+----------+-----+



#### Most frequent host:

In [38]:
top_host = weblogs.groupBy("requesting_host").count().orderBy(desc("count")).limit(1)

In [39]:
top_host.show()

+------------------+-----+
|   requesting_host|count|
+------------------+-----+
|edams.ksc.nasa.gov| 6530|
+------------------+-----+



#### Hours with the most traffic:

In [40]:
hour_traffic = weblogs.withColumn("hour",hour("datetime")).groupBy("hour").count().orderBy(desc("count"))

In [41]:
hour_traffic.show()

+----+------+
|hour| count|
+----+------+
|  15|109461|
|  12|105143|
|  13|104535|
|  14|101393|
|  16| 99527|
|  11| 95340|
|  10| 88309|
|  17| 80834|
|   9| 78695|
|  18| 66809|
|   8| 65443|
|  22| 60673|
|  20| 59944|
|  19| 59315|
|  21| 57985|
|  23| 54570|
|   0| 47862|
|   7| 47386|
|   1| 38531|
|   2| 32508|
+----+------+
only showing top 20 rows



#### Number of 404 erros each day:

In [42]:
error_day  = weblogs.where(col("status") == 404).withColumn("date",to_date("datetime")).groupBy("date").count().orderBy(desc("count"))

In [43]:
error_day.show()

+----------+-----+
|      date|count|
+----------+-----+
|1995-08-30|  571|
|1995-08-07|  537|
|1995-08-31|  526|
|1995-08-24|  420|
|1995-08-29|  420|
|1995-08-25|  415|
|1995-08-28|  410|
|1995-08-08|  391|
|1995-08-06|  373|
|1995-08-27|  370|
|1995-08-26|  366|
|1995-08-04|  346|
|1995-08-23|  345|
|1995-08-15|  327|
|1995-08-10|  315|
|1995-08-20|  312|
|1995-08-21|  305|
|1995-08-03|  303|
|1995-08-22|  288|
|1995-08-14|  287|
+----------+-----+
only showing top 20 rows

